# FAISS vector store db

Facebook AI Similarity Search (FAISS) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contsains supporting code for evalution and parameter tuning.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader  = TextLoader("speech.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
final_docs = text_splitter.split_documents(documents)


In [6]:
embeddings = OllamaEmbeddings(model = 'gemma:2b')
# if you dont specify any model then by default it consider the model as llam2
db = FAISS.from_documents(final_docs, embeddings)
db

In [7]:
## queryng

query = "what is the name of the Ex Chief Minister of Andhra Pradesh?"

retrived_result  = db.similarity_search(query)

In [10]:
retrived_result[0].page_content

'In the 2019 National and State election held in April and May 2019, YSR Congress Party swept the polls and won 151 of the total 175 assembly seats\nand 22 of the 25 Lok Sabha seats in Andhra Pradesh. He took oath as the Chief Minister on 30 May 2019.[52] His chief ministership has been marked \nby a slew of welfare schemes such as Jagananna Amma Vodi, Navaratnalu.[53] Jagananna Amma Vodi provides financial assistance for mothers or \nguardians from the Below Poverty Line, to educate their children.[54][55] Navaratnalu is a collection of nine welfare schemes covering farmers, \nwomen, medical and health, education and Special Category Status.[53] He scrapped the plans for a new capital at Amaravati, proposed by the former\nTDP government, and has proposed three different capitals for the judicial, administrative and legislative branches at Kurnool, Amaravati and \nVisakhapatnam respectively.[56] The proposal resulted in widespread protests by the farmers of Amaravati.[57] The Andhra Pr

# As a Retriver

we can also convert the vectorstore into a Retriver class. This allows us to easily use it in other LangChain methods, which largely work with retrivers.

To use any vector db into any other langchain methods we first need to convert into A Retriver usning as_retriver() function and then we can call it in othe langchain methods.

In [12]:
retriver = db.as_retriever()

retriver.invoke(query)

[Document(metadata={'source': 'speech.txt'}, page_content='In the 2019 National and State election held in April and May 2019, YSR Congress Party swept the polls and won 151 of the total 175 assembly seats\nand 22 of the 25 Lok Sabha seats in Andhra Pradesh. He took oath as the Chief Minister on 30 May 2019.[52] His chief ministership has been marked \nby a slew of welfare schemes such as Jagananna Amma Vodi, Navaratnalu.[53] Jagananna Amma Vodi provides financial assistance for mothers or \nguardians from the Below Poverty Line, to educate their children.[54][55] Navaratnalu is a collection of nine welfare schemes covering farmers, \nwomen, medical and health, education and Special Category Status.[53] He scrapped the plans for a new capital at Amaravati, proposed by the former\nTDP government, and has proposed three different capitals for the judicial, administrative and legislative branches at Kurnool, Amaravati and \nVisakhapatnam respectively.[56] The proposal resulted in widespre

# Similarity search with score

There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2(Manhattan) distance. Therefore a lower score means more similar.


In [14]:
docs_and_Score =  db.similarity_search_with_score(query)
docs_and_Score

[(Document(metadata={'source': 'speech.txt'}, page_content='In the 2019 National and State election held in April and May 2019, YSR Congress Party swept the polls and won 151 of the total 175 assembly seats\nand 22 of the 25 Lok Sabha seats in Andhra Pradesh. He took oath as the Chief Minister on 30 May 2019.[52] His chief ministership has been marked \nby a slew of welfare schemes such as Jagananna Amma Vodi, Navaratnalu.[53] Jagananna Amma Vodi provides financial assistance for mothers or \nguardians from the Below Poverty Line, to educate their children.[54][55] Navaratnalu is a collection of nine welfare schemes covering farmers, \nwomen, medical and health, education and Special Category Status.[53] He scrapped the plans for a new capital at Amaravati, proposed by the former\nTDP government, and has proposed three different capitals for the judicial, administrative and legislative branches at Kurnool, Amaravati and \nVisakhapatnam respectively.[56] The proposal resulted in widespr

we can also convert the query into embedding vector and use that vector to query the db with `similarity_search_by_vector()` method.

In [15]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.350779265165329,
 0.08504191786050797,
 -0.41835856437683105,
 1.76663339138031,
 1.6184475421905518,
 2.1118364334106445,
 0.4129340648651123,
 -0.9951862096786499,
 0.7701394557952881,
 -0.448603093624115,
 0.6652577519416809,
 -1.2257391214370728,
 -0.38182517886161804,
 1.8993481397628784,
 0.05438195541501045,
 -1.408180594444275,
 4.413474082946777,
 2.5154178142547607,
 -0.9824066758155823,
 0.553705632686615,
 2.6260907649993896,
 -0.6252179145812988,
 -0.014814337715506554,
 -0.14468218386173248,
 -0.993025004863739,
 0.019656846299767494,
 1.291729211807251,
 -0.23945827782154083,
 0.3199138641357422,
 -2.8043372631073,
 -0.2557128071784973,
 0.5640994906425476,
 -0.25522786378860474,
 -1.6134542226791382,
 0.24143780767917633,
 0.4851090908050537,
 -0.6965349912643433,
 -2.178056478500366,
 3.285900354385376,
 -0.6913362145423889,
 -0.31118887662887573,
 -0.5187168121337891,
 1.6515421867370605,
 0.026240622624754906,
 1.3347407579421997,
 0.919305145740509,
 1.130384087

In [16]:
docs_and_score_vector = db.similarity_search_by_vector(embedding_vector)

docs_and_score_vector

[Document(metadata={'source': 'speech.txt'}, page_content='In the 2019 National and State election held in April and May 2019, YSR Congress Party swept the polls and won 151 of the total 175 assembly seats\nand 22 of the 25 Lok Sabha seats in Andhra Pradesh. He took oath as the Chief Minister on 30 May 2019.[52] His chief ministership has been marked \nby a slew of welfare schemes such as Jagananna Amma Vodi, Navaratnalu.[53] Jagananna Amma Vodi provides financial assistance for mothers or \nguardians from the Below Poverty Line, to educate their children.[54][55] Navaratnalu is a collection of nine welfare schemes covering farmers, \nwomen, medical and health, education and Special Category Status.[53] He scrapped the plans for a new capital at Amaravati, proposed by the former\nTDP government, and has proposed three different capitals for the judicial, administrative and legislative branches at Kurnool, Amaravati and \nVisakhapatnam respectively.[56] The proposal resulted in widespre

In [18]:
### Saving and loading the vector db
## the vector store db gets saved as pikle file

db.save_local("faiss_index")

when you are trying to load the vector db from local and if it's throwing a value error then you need to pass `allow_dangerous_deserialization = True ` as a parameter

In [21]:
## loading the saved vector db from local

new_db = FAISS.load_local('faiss_index', embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [22]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='In the 2019 National and State election held in April and May 2019, YSR Congress Party swept the polls and won 151 of the total 175 assembly seats\nand 22 of the 25 Lok Sabha seats in Andhra Pradesh. He took oath as the Chief Minister on 30 May 2019.[52] His chief ministership has been marked \nby a slew of welfare schemes such as Jagananna Amma Vodi, Navaratnalu.[53] Jagananna Amma Vodi provides financial assistance for mothers or \nguardians from the Below Poverty Line, to educate their children.[54][55] Navaratnalu is a collection of nine welfare schemes covering farmers, \nwomen, medical and health, education and Special Category Status.[53] He scrapped the plans for a new capital at Amaravati, proposed by the former\nTDP government, and has proposed three different capitals for the judicial, administrative and legislative branches at Kurnool, Amaravati and \nVisakhapatnam respectively.[56] The proposal resulted in widespre